In [2]:
import pandas as pd

In [145]:
dir_optOut = "../data/optOut/"
dir_optIn = "../data/optIn/"

In [146]:
import os

def load_dir(dir):
    dfs = []
    for file_name in os.listdir(dir):
        file = os.path.join(dir, file_name)
        if file.endswith(".json") and file_name != "metadata.json":
            temp = pd.read_json(file, typ='series', orient='records')
            columns = []
            values = []

            for item in temp.items():
                if not item[0] == 'data':
                    columns.append(item[0])
                    values.append(item[1])
            for item in temp['data']:
                columns.append(item)
                values.append(temp['data'].get(item))

            additional_columns = ['contains_data_blob', 'contains_FormData', 'contains_Object', 'contains_SVGSVGElement']
            default_values = [False]*len(additional_columns)
            columns += additional_columns
            values += default_values
            df = pd.DataFrame([values], columns=columns)
            #df.set_index('initialUrl')
            dfs.append(df)
    
    temp = pd.concat(dfs, ignore_index=True)
    return temp

dfs_optIn = load_dir(dir_optIn)
dfs_optIn.head(5)

,initialUrl,finalUrl,timeout,testStarted,testFinished,reload,cmps,screenshots,requests,cookies,apis,targets,contains_data_blob,contains_FormData,contains_Object,contains_SVGSVGElement
0,http://tradplusad.com/,https://www.tradplusad.com/,True,1698600172591,1698600207186,reloaded page,[],data/optIn/tradplusad.com_fb11.jpg,"[{'url': 'http://tradplusad.com/', 'method': '...",[],"{'callStats': {}, 'savedCalls': []}","[{'type': 'page', 'url': 'http://tradplusad.co...",False,False,False,False
1,http://alexa.com/,https://alexa.com/,False,1698597502076,1698597521144,reloaded page,[],data/optIn/alexa.com_3200.jpg,"[{'url': 'http://alexa.com/', 'method': 'GET',...",[],"{'callStats': {}, 'savedCalls': []}","[{'type': 'page', 'url': 'http://alexa.com/'}]",False,False,False,False
2,http://bstatic.com/,https://www.booking.com/index.en-gb.html?label...,True,1698599447327,1698599480650,reloaded page,"[{'final': True, 'name': 'Onetrust', 'open': T...",data/optIn/bstatic.com_f6fb.jpg,"[{'url': 'http://bstatic.com/', 'method': 'GET...","[{'name': '_GRECAPTCHA', 'domain': 'www.google...",{'callStats': {'https://cf.bstatic.com/static/...,"[{'type': 'page', 'url': 'http://bstatic.com/'...",False,False,False,False
3,http://aniwatch.to/,https://aniwatch.to/,True,1698594011127,1698594045722,reloaded page,[],data/optIn/aniwatch.to_714b.jpg,"[{'url': 'http://aniwatch.to/', 'method': 'GET...","[{'name': '_ga', 'domain': '.aniwatch.to', 'pa...",{'callStats': {'https://www.googletagmanager.c...,"[{'type': 'page', 'url': 'http://aniwatch.to/'...",False,False,False,False
4,http://poste.it/,https://poste.it/,True,1698600575078,1698600609688,reloaded page,[],data/optIn/poste.it_e385.jpg,"[{'url': 'http://poste.it/', 'method': 'GET', ...",[{'name': 'AMCV_3A3C1D085C5C70990A495CFC%40Ado...,"{'callStats': {}, 'savedCalls': []}","[{'type': 'page', 'url': 'http://poste.it/'}]",False,False,False,False


In [148]:
other = 0
for index, row in dfs_optIn.iterrows():
    for savedCall in row['apis']['savedCalls']:
        for argument in savedCall['arguments']:
            if argument == '[object Blob]':
                dfs_optIn.at[index, 'contains_data_blob'] = True
            if argument == '[object FormData]':
                dfs_optIn.at[index, 'contains_FormData'] = True
            if argument == '[object Object]':
                dfs_optIn.at[index, 'contains_Object'] = True
            if argument == '[object SVGSVGElement]':
                dfs_optIn.at[index, 'contains_SVGSVGElement'] = True
            if argument.startswith('[object '):
                if True in {row['contains_data_blob'], row['contains_FormData'], row['contains_Object'], row['contains_SVGSVGElement']}:
                    continue
                other += 1
                print(argument)

print(dfs_optIn['contains_data_blob'].value_counts())
print("\n")
print(dfs_optIn['contains_FormData'].value_counts())
print("\n")
print(dfs_optIn['contains_Object'].value_counts())
print("\n")
print(dfs_optIn['contains_SVGSVGElement'].value_counts())
print("\n")
print('number of other object types found: ' + str(other))

contains_data_blob
False    7374
True      665
Name: count, dtype: int64


contains_FormData
False    7943
True       96
Name: count, dtype: int64


contains_Object
False    8023
True       16
Name: count, dtype: int64


contains_SVGSVGElement
False    8038
True        1
Name: count, dtype: int64


number of other object types found: 0
